In [27]:
from pyhive import presto
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

from datetime import datetime, timedelta

import seaborn as sns

conn = presto.connect(
    host='presto.processing.yoda.run', 
    ## presto.processing.yoda.run
    ## bi-presto.serving.data.production.internal
    port=80,
    username='manoj.ravirajan@rapido.bike'
)

In [28]:
query = f"""

with base as (
select 
    distinct *
from experiments.customer_referral_fraud_22
) 

select 
    referreruserid parent,
    referreduserid child 
from base
/*where referreruserid in ('62c920a53159e3064b05383b',
'6303c1f404cb1bab590dd221',
'6303c25404cb1b4fb00dd272',
'6303c312e1dabc146cd00662',
'6303c37b04cb1b5a740dd36a',
'6303c3eee1dabcd1a9d0070c',
'6303c46e04cb1b56ac0dd433',
'6303c4cc51f7143230631ac7',
'6303c59751f7149e16631b6e',
'630769c16279cd6d93e45575',
'631357a62c7ad8246e65efea'
)
*/
--limit 100
"""

In [29]:
df = pd.read_sql(query, conn)
df

,parent,child
0,6238266da63f417352d858cd,635fe8fbb087cd17881e394c
1,62947ddb98c7502d8d57b978,635f32ceae48a7c799d32984
2,623bee76734ac0ad7c55dc48,635f262185b399ce7c55c4ae
3,623f3684b9a21e5507cebe19,635fe5057ca17a2dd108a712
4,62ad8ce97e5fa08f8f8043fe,635f8bb7ae48a72067d36ee9
...,...,...
904522,62d37e17082b672904b408e5,62f387cd42eb0adca2d77966
904523,626f99d9edb0a6e939c9d522,630de761fb9d0c55ec96410c
904524,62f87b0f382924fa4ae24115,62f913ed83af2fdd4fe13540
904525,625bf5fd8542166d3e0b80f1,62f91266cfaed115c4d9fc59


In [30]:
def get_ancestry_dataframe_flat(df):

    def get_child_list(parent_id):

        list_of_children = list()
        list_of_children.append(df[df['parent'] == parent_id]['child'].values)

        for i, r in df[df['parent'] == parent_id].iterrows():
            if r['child'] != parent_id:
                list_of_children.append(get_child_list(r['child']))

        # flatten list
        list_of_children = [item for sublist in list_of_children for item in sublist]
        return list_of_children

    new_df = pd.DataFrame(columns=['descendant', 'ancestor']).astype(int)
    for index, row in df.iterrows():
        temp_df = pd.DataFrame(columns=['descendant', 'ancestor'])
        temp_df['descendant'] = pd.Series(get_child_list(row['parent']))
        temp_df['ancestor'] = row['parent']
        new_df = new_df.append(temp_df)

    new_df = new_df\
        .drop_duplicates()\
        .sort_values(['ancestor', 'descendant'])\
        .reset_index(drop=True)

    return new_df

In [31]:
get_ancestry_dataframe_flat(df)

In [26]:
get_ancestry_dataframe_flat(df)

,descendant,ancestor
0,6303c1f404cb1bab590dd221,62c920a53159e3064b05383b
1,6303c25404cb1b4fb00dd272,62c920a53159e3064b05383b
2,6303c312e1dabc146cd00662,62c920a53159e3064b05383b
3,6303c37b04cb1b5a740dd36a,62c920a53159e3064b05383b
4,6303c3eee1dabcd1a9d0070c,62c920a53159e3064b05383b
...,...,...
81,63076a29c2274a27a112b8da,630769c16279cd6d93e45575
82,631357a62c7ad8246e65efea,630769c16279cd6d93e45575
83,63135841e8a740a2495b1514,630769c16279cd6d93e45575
84,6355343c85b399d6f34e90b2,630769c16279cd6d93e45575


In [4]:
import numpy as np
import pandas as pd  # only used to return a dataframe


def list_ancestors(edges):
    """
    Take edge list of a rooted tree as a numpy array with shape (E, 2),
    child nodes in edges[:, 0], parent nodes in edges[:, 1]
    Return pandas dataframe of all descendant/ancestor node pairs

    Ex:
        df = pd.DataFrame({'child': [200, 201, 300, 301, 302, 400],
                           'parent': [100, 100, 200, 200, 201, 300]})

        df
           child  parent
        0    200     100
        1    201     100
        2    300     200
        3    301     200
        4    302     201
        5    400     300

        list_ancestors(df.values)

        returns

            descendant  ancestor
        0          200       100
        1          201       100
        2          300       200
        3          300       100
        4          301       200
        5          301       100
        6          302       201
        7          302       100
        8          400       300
        9          400       200
        10         400       100
    """
    ancestors = []
    for ar in trace_nodes(edges):
        ancestors.append(np.c_[np.repeat(ar[:, 0], ar.shape[1]-1),
                               ar[:, 1:].flatten()])
    return pd.DataFrame(np.concatenate(ancestors),
                        columns=['descendant', 'ancestor'])


def trace_nodes(edges):
    """
    Take edge list of a rooted tree as a numpy array with shape (E, 2),
    child nodes in edges[:, 0], parent nodes in edges[:, 1]
    Yield numpy array with cross-section of tree and associated
    ancestor nodes

    Ex:
        df = pd.DataFrame({'child': [200, 201, 300, 301, 302, 400],
                           'parent': [100, 100, 200, 200, 201, 300]})

        df
           child  parent
        0    200     100
        1    201     100
        2    300     200
        3    301     200
        4    302     201
        5    400     300

        trace_nodes(df.values)

        yields

        array([[200, 100],
               [201, 100]])

        array([[300, 200, 100],
               [301, 200, 100],
               [302, 201, 100]])

        array([[400, 300, 200, 100]])
    """
    mask = np.in1d(edges[:, 1], edges[:, 0])
    gen_branches = edges[~mask]
    edges = edges[mask]
    yield gen_branches
    while edges.size != 0:
        mask = np.in1d(edges[:, 1], edges[:, 0])
        next_gen = edges[~mask]
        gen_branches = numpy_col_inner_many_to_one_join(next_gen, gen_branches)
        edges = edges[mask]
        yield gen_branches


def numpy_col_inner_many_to_one_join(ar1, ar2):
    """
    Take two 2-d numpy arrays ar1 and ar2,
    with no duplicate values in first column of ar2
    Return inner join of ar1 and ar2 on
    last column of ar1, first column of ar2

    Ex:

        ar1 = np.array([[1,  2,  3],
                        [4,  5,  3],
                        [6,  7,  8],
                        [9, 10, 11]])

        ar2 = np.array([[ 1,  2],
                        [ 3,  4],
                        [ 5,  6],
                        [ 7,  8],
                        [ 9, 10],
                        [11, 12]])

        numpy_col_inner_many_to_one_join(ar1, ar2)

        returns

        array([[ 1,  2,  3,  4],
               [ 4,  5,  3,  4],
               [ 9, 10, 11, 12]])
    """
    ar1 = ar1[np.in1d(ar1[:, -1], ar2[:, 0])]
    ar2 = ar2[np.in1d(ar2[:, 0], ar1[:, -1])]
    if 'int' in ar1.dtype.name and ar1[:, -1].min() >= 0:
        bins = np.bincount(ar1[:, -1])
        counts = bins[bins.nonzero()[0]]
    else:
        counts = np.unique(ar1[:, -1], False, False, True)[1]
    left = ar1[ar1[:, -1].argsort()]
    right = ar2[ar2[:, 0].argsort()]
    return np.concatenate([left[:, :-1],
                           right[np.repeat(np.arange(right.shape[0]),
                                           counts)]], 1)